Set-up

In [64]:
import requests
import pandas as pd
import numpy as np
import urllib.request

Function to calculate distance given latitude and longitude

In [65]:
from math import radians, cos, sin, asin, sqrt

def haversine(lon1, lat1, lon2, lat2):

    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371 # Radius of earth in kilometers. Use 3956 for miles. Determines return value units.
    return c * r *1000

Get LTA camera id and images

In [66]:
traffic_image_url='http://datamall2.mytransport.sg/ltaodataservice/Traffic-Imagesv2'
headers_val={'AccountKey':'AO4qMbK3S7CWKSlplQZqlA=='}
traffic_image_req=requests.get(url=traffic_image_url,headers=headers_val)
traffic_image_df=pd.DataFrame(eval(traffic_image_req.content)['value'])
traffic_image_df['Count']=np.random.uniform(low=0, high=20, size=(len(traffic_image_df.index),)).astype(int)
traffic_image_df['is_jam']=0
traffic_image_df=traffic_image_df.merge(pd.read_csv('traffic_camera_region_RoadName.csv',converters={'CameraID':str}),'left','CameraID')

Get LTA incidents on Expressways

In [68]:
traffic_incidents_url='http://datamall2.mytransport.sg/ltaodataservice/TrafficIncidents'
traffic_incidents_req=requests.get(url=traffic_incidents_url,headers=headers_val)
traffic_incidents_df=pd.DataFrame(eval(traffic_incidents_req.content)['value'])
incidents_roads=['AYE','BKE','CTE','ECP','KJE','KPE','MCE','PIE','SLE','TPE','Sentosa','Tuas','Woodlands']
traffic_incidents_df=traffic_incidents_df[traffic_incidents_df['Message'].apply(lambda x: any(expressway in x for expressway in incidents_roads))]

NEA API to get rainfall in mm

In [70]:
weatherreq=requests.get(url='https://api.data.gov.sg/v1/environment/rainfall')
weather_df=pd.DataFrame(eval(weatherreq.content)['metadata']['stations'])

weather_df['latitude']=weather_df['location'].apply(lambda x: x['latitude'])
weather_df['longitude']=weather_df['location'].apply(lambda x: x['longitude'])
weather_df['timestamp']=eval(weatherreq.content)['items'][0]['timestamp']
weather_df['timestamp']=pd.to_datetime(weather_df['timestamp'])

station_rainfall=pd.DataFrame(eval(weatherreq.content)['items'][0]['readings']).rename(columns={'value':'rainfall'})

weather_df=weather_df.merge(station_rainfall,how='left',left_on='id',right_on='station_id')
weather_df=weather_df.drop(['id','device_id','station_id','location'],axis=1)

Calculations and table joining


In [71]:
traffic_image_df['key']=0
traffic_incidents_df['key']=0
weather_df['key']=0

Select incidents that occur within 500m of a camera location

In [72]:
nearest_incidents=traffic_image_df.merge(traffic_incidents_df,'outer','key')
nearest_incidents['incident_distance_from_id']=(np.vectorize(haversine)(nearest_incidents['Latitude_x'],nearest_incidents['Longitude_x'],nearest_incidents['Latitude_y'],nearest_incidents['Longitude_y']))
nearest_incidents=nearest_incidents[nearest_incidents['incident_distance_from_id']<500].sort_values('incident_distance_from_id')
nearest_incidents=nearest_incidents[['CameraID','Message']]

Select nearest weather station for each camera id

In [73]:
final_df=traffic_image_df.merge(weather_df,'outer','key')
final_df['distance_from_id']=(np.vectorize(haversine)(final_df['Latitude'],final_df['Longitude'],final_df['latitude'],final_df['longitude']))
final_df=final_df.sort_values('distance_from_id').groupby('CameraID').head(1)[['CameraID','Latitude','Longitude','Region','rainfall','ImageLink','RoadName','Count','is_jam']]
final_df=final_df.sort_values('CameraID').reset_index(drop=True)


Convert dataframes to CSV to be used in frontend

In [74]:
final_df.to_csv('main_df.csv',index=False)
nearest_incidents.to_csv('traffic_incidents.csv',index=False)

# END

In [1]:
import pandas as pd

In [ ]:
main_df=pd.read_csv('main_df.csv')
camera = []
for i in range(len(main_df)):
    d=dict(name = main_df.iloc[i,6], lat = main_df.iloc[i,1], lon = main_df.iloc[i,2])
    camera.append(d)
camera

In [10]:
# Create drop down options.
dd_options = [dict(value=c["name"], label=c["name"]+'_') for c in camera]
dd_defaults = [o["value"] for o in dd_options]

In [5]:
relevant_camera_options=main_df[['CameraID', 'RoadName']].values.tolist()
relevant_camera_options

[[1001, 'East Coast Parkway (ECP)'],
 [1002, 'Pan-Island Expressway (PIE)'],
 [1003, 'Pan-Island Expressway (PIE)'],
 [1004, 'Kallang Way Flyover'],
 [1005, 'Kallang-Paya Lebar Expressway (KPE)'],
 [1006, 'Kallang-Paya Lebar Expressway (KPE)'],
 [1501, 'Marina Coastal Expressway (MCE)'],
 [1502, 'Marina Coastal Expressway (MCE)'],
 [1503, 'Marina Coastal Expressway (MCE)'],
 [1504, 'Marina Coastal Expressway (MCE)'],
 [1505, 'Marina Coastal Expressway (MCE)'],
 [1701, 'Central Expressway (CTE)'],
 [1702, 'Central Expressway (CTE)'],
 [1703, 'Central Expressway (CTE)'],
 [1704, 'Central Expressway (CTE)'],
 [1705, 'Central Expressway (CTE)'],
 [1706, 'Central Expressway (CTE)'],
 [1707, 'Central Expressway (CTE)'],
 [1709, 'Central Expressway (CTE)'],
 [1711, 'Central Expressway (CTE)'],
 [2701, 'Woodlands Causeway'],
 [2702, 'Woodlands Checkpoint'],
 [2703, 'Pan-Island Expressway (PIE)'],
 [2704, 'Bukit Timah Expressway (BKE)'],
 [2705, 'Bukit Timah Expressway (BKE)'],
 [2706, 'Bukit T

In [10]:
formatted_relevant_camera_options = [{'label':x[1]+'_'+str(x[0]), 'value':x[0]} for x in relevant_camera_options]
formatted_relevant_camera_options

[{'label': 'East Coast Parkway (ECP)_1001', 'value': 1001},
 {'label': 'Pan-Island Expressway (PIE)_1002', 'value': 1002},
 {'label': 'Pan-Island Expressway (PIE)_1003', 'value': 1003},
 {'label': 'Kallang Way Flyover_1004', 'value': 1004},
 {'label': 'Kallang-Paya Lebar Expressway (KPE)_1005', 'value': 1005},
 {'label': 'Kallang-Paya Lebar Expressway (KPE)_1006', 'value': 1006},
 {'label': 'Marina Coastal Expressway (MCE)_1501', 'value': 1501},
 {'label': 'Marina Coastal Expressway (MCE)_1502', 'value': 1502},
 {'label': 'Marina Coastal Expressway (MCE)_1503', 'value': 1503},
 {'label': 'Marina Coastal Expressway (MCE)_1504', 'value': 1504},
 {'label': 'Marina Coastal Expressway (MCE)_1505', 'value': 1505},
 {'label': 'Central Expressway (CTE)_1701', 'value': 1701},
 {'label': 'Central Expressway (CTE)_1702', 'value': 1702},
 {'label': 'Central Expressway (CTE)_1703', 'value': 1703},
 {'label': 'Central Expressway (CTE)_1704', 'value': 1704},
 {'label': 'Central Expressway (CTE)_1705'